In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
from sklearn.datasets import load_diabetes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

description of dataset [here](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset)

In [7]:
dataset = load_diabetes()

In [8]:
columns = [
    'age',
    'sex',
    'body_mass_index',
    'average_blood_pressure',
    's1',
    's2',
    's3',
    's4',
    's5',
    's6'
]

In [9]:
X = dataset['data']
y = dataset['target']

In [10]:
X.shape

(442, 10)

In [11]:
df_x = pd.DataFrame(X, columns=columns)
df_x.describe()

,age,sex,body_mass_index,average_blood_pressure,s1,s2,s3,s4,s5,s6
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02
mean,-3.639623e-16,1.309912e-16,-8.013951e-16,1.289818e-16,-9.042540e-17,1.301121e-16,-4.563971e-16,3.863174e-16,-3.848103e-16,-3.398488e-16
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123996e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260974e-01,-1.377672e-01
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665645e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324879e-02,-3.317903e-02
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670611e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947634e-03,-1.077698e-03
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564384e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243323e-02,2.791705e-02
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320442e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335990e-01,1.356118e-01


# Solving Normally

Cross Validating. 
We will use `KFold` class to split the data.

In [12]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

In [13]:
def rmse(y_true, y_hat):
    # TODO implement rmse
    return np.sqrt(mean_squared_error(y_true, y_hat))

def print_all_metrics(y_true, y_hat, type='train'):
    print('''
    ....{} metrics....
    rmse: {:.2f}
    mae: {:.2f},
    r2: {:.2f}
    '''.format(
        type,
        rmse(y_true, y_hat),
        mean_absolute_error(y_true, y_hat),
        r2_score(y_true, y_hat)
    ))
    
def all_metrics(y_true, y_hat):
    return (
        rmse(y_true, y_hat),
        mean_absolute_error(y_true, y_hat),
        r2_score(y_true, y_hat) 
    )

In [14]:
random_state = 142
folds = KFold(5, shuffle=True, random_state=random_state)

In [15]:
def do_cv(folds, mdl, X, y):
    cv_metrics = list()
    # TODO impl
    
    for cv_idx, (train_index, test_index) in enumerate(folds.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        m = mdl.fit(X_train, y_train)
        y_pred = m.predict(X_test)
        _rmse, _mae, _r2 = all_metrics(y_test, y_pred)
        
        cv_metrics.append([cv_idx, _rmse, _mae, _r2])
        
    return pd.DataFrame(cv_metrics, columns=['cv_idx', 'rmse', 'mae', 'r2'])

In [16]:
cv_metrics = do_cv(folds, LinearRegression(), X, y)

cv_metrics

,cv_idx,rmse,mae,r2
0,0,47.452700,38.011039,0.512996
1,1,54.830096,43.961871,0.515797
2,2,56.735936,46.679995,0.430081
3,3,56.915700,47.203529,0.495173
4,4,56.144112,45.848717,0.507949


# Bagging:  Bootstrap Aggregating

## Main Idea

Bagging could be done by simply **train several weak predictor and take the average of each predictors**.

Bagging comes from name: *Bootstrap Aggregating*. Bootstraping in statistics is an estimation of sampling distribution by taking random samples with replacement$^{5}$. Bagging predictor uses *bootstrap* because it literally train on different algorithm with same data${^6}$, thus it is sampling with replacement / bootstraping.


> A critical factor in whether bagging will improve accuracy is the stability of the procedure for constructing $\varphi$. If changes in $\mathcal{L}$  i.e. a replicate $\mathcal{L}$, produces small changes in $\varphi$, then $\varphi_B$ will be close to $\varphi$. Improvement will occur for unstable procedures where a small change in $\mathcal{L}$ can result in large changes in $\varphi$ $^6$

where:

- $\mathcal{L}$ is the learning sets / training set / training data
- $\varphi$ is the predictor
- $\varphi_B$ is the predictor trained on bootstrap sample of data



What does it means? It means that Bagging will excel if there is a small change in training set $\mathcal{L}$, results in big big change in prediction of $\varphi$.


## What Algorithms Could be Combined by Bagging?

> neural nets, classification and regression trees, and subset selection


## Toy Regressors

For this purpose we will use 3 toy regressors: `LinearRegression`, `Lasso`, and `Ridge` 

$$
f(x) =  \sum_{m=0}^{M} \frac{1}{M} * h_m(x)
$$

where:

- $m$ is the model index

- $h(x)$ is the prediction for model $h_m$ given the input $x$

- $f(x)$ is the final prediction

## Simple Bagging Implementation

In [17]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.base import BaseEstimator, RegressorMixin
from copy import deepcopy

In [18]:
class SimpleBaggingModel(BaseEstimator, RegressorMixin):
    def __init__(self, base_estimator, n_iter, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.models = [base_estimator] * n_iter # TODO
        self.len_models = len(self.models)
        
        self.model_weights = [1. / n_iter for i in range(n_iter)] # TODO
    
    def _get_random_choice(self, X, y):
        # TODO 
        idxs = np.random.choice(np.arange(X.shape[0]), X.shape[0])
        X_choice = X[idxs]
        y_choice = y[idxs]
        return X_choice, y_choice
        
    def fit(self, X, y):
        # TODO 
        for i in range(self.len_models):
            X_train, y_train = self._get_random_choice(X, y)
            self.models[i].fit(X_train, y_train) ## can be simplified to self.models[i].fit(*self._get_random_choice(X, y))
        
        return self
    
    def predict(self, X):
        predictions = list()
        # TODO 
        
        for i in range(self.len_models):
            y_pred = self.models[i].predict(X)
            predictions.append(y_pred)
        
        return np.sum(
            (np.array(predictions).T * self.model_weights).T,
            axis=0
        )
        

In [19]:
do_cv(folds, SimpleBaggingModel(LinearRegression(), 10), X, y)

,cv_idx,rmse,mae,r2
0,0,49.827066,40.499336,0.463041
1,1,54.359228,42.743706,0.524078
2,2,57.711236,47.095899,0.410318
3,3,56.865259,46.960065,0.496067
4,4,55.425224,45.305436,0.520469


## Bagging Using Sklearn

Sklearn has a nice wrapper for simple bagging, both classifier and regressor. It resides at `sklearn.ensemble` module, `BaggingClassifier`, `BaggingRegressor`, `VotingClassifier` and `VotingRegressor`. 

In [22]:
from sklearn.ensemble import VotingRegressor, BaggingRegressor

In [26]:
# TODO: do_cv with BaggingRegressor, base_estimator = LinearRegression
do_cv(folds, BaggingRegressor(base_estimator=LinearRegression()), X, y)

,cv_idx,rmse,mae,r2
0,0,48.497070,39.111117,0.491324
1,1,54.933590,44.000295,0.513967
2,2,56.873263,46.557978,0.427318
3,3,56.897640,47.119511,0.495493
4,4,56.379280,45.924513,0.503819


In [40]:
# TODO: do_cv with VotingRegressor, models: LinearRegression, Lasso, Ridge
do_cv(folds, VotingRegressor(
    estimators=[
        ('linreg', LinearRegression()),
        ('lasso', Lasso()),
        ('ridge', Ridge())
    ]), 
      X, 
      y
     )

,cv_idx,rmse,mae,r2
0,0,48.217676,38.881483,0.497168
1,1,58.479110,49.415773,0.449204
2,2,59.139107,50.065281,0.380778
3,3,59.039757,48.936453,0.456790
4,4,58.301086,49.801070,0.469415


# Boosting
There are 2 types of popular boosting: AdaBoost and Gradient Boosting, where each boosting is based on decision tree.

## Adaptive Boosting (AdaBoost)

### Main Idea
Main idea for Adaptive Boosting is 
> iteratively train $T$ models, for each model $t \in T$: train the model with **weighted samples**, where samples with **less error will receive less weight** in next iteration. Then combine the prediction of each model $t$ with **weighted estimators**.

### Deeper Understanding: the Algorithm
Algorithm for AdaBoost is given by $^{1, 2}$, and simple implementation could be also be found online $^{3}$. 

Given $(x_1, y_1) ... (x_m, y_m)$ where $x_i \in \mathcal{X}$, $y_i \in \{-1, +1\}$

Initialize: $D_1(i) = 1/m$ for $i = 1, ..., m.$ 

For $t = 1...T$

- Train weak learner using distribution $D_t$
- Get weak hypothesis $h_t: \mathcal{X} -> \{-1, +1\}$
- Choose $\alpha = \frac{1}{2} \text{ ln } \frac{(1 - \epsilon_t)}{\epsilon_t}$
- Update, for $i \text{ = 1, ... , m}$:

$$
D_{t+1}(i) = \frac{D_t(i)\text{ exp }(-\alpha_t y_i h_t (x_i))} {Z_t}
$$

Where $Z_t$ is normalization factor. Final output:

$$
H(x) = \text{sign}(\sum_{t=1}^{T} \alpha_t h_t(x))
$$

However this algorithm is for training a classifier, for regressor refer to [4]


Notes:
> - D = sample weight --> weight for each sample
> - Sample in which model performs badly will be weighted more than other sample
> - $\alpha$ = estimator weight

In [41]:
from sklearn.ensemble import AdaBoostRegressor

In [42]:
# TODO define AdaBoostRegressor, n_estimators=10
mdl = AdaBoostRegressor(base_estimator=LinearRegression(),
                        n_estimators=10)

In [43]:
# TODO: do_cv with AdaBoostRegressor
do_cv(folds, mdl, X, y)

,cv_idx,rmse,mae,r2
0,0,46.680379,37.186314,0.528720
1,1,55.343572,44.372483,0.506686
2,2,57.396501,47.280182,0.416733
3,3,56.732209,47.179074,0.498423
4,4,56.775966,46.935917,0.496812


Be careful of using Lasso or Ridge (and other regularized model) for boosting. The results might be worse, or only slightly better.

In [48]:
# comparison between Lasso vs AdaBoost with Lasso as estimator
do_cv(folds, Lasso(), X, y).mean()

cv_idx     2.000000
rmse      61.887992
mae       52.837713
r2         0.344447
dtype: float64

In [49]:
mdl_lasso = AdaBoostRegressor(base_estimator=Lasso(),
                        n_estimators=10)

do_cv(folds, mdl_lasso, X, y).mean()

cv_idx     2.000000
rmse      60.291436
mae       51.311308
r2         0.377728
dtype: float64

## Gradient Boosting (GBM)

Gradient boosting works slightly different from Adaptive Boosting. Main differences between those two algorithm are: **how to fit each iteration model** and **how to determine estimator weights for each iteration**. Idea comes from Friedman$^{9}$, and now have been improved into *extreme* gradient boosting, for example: LightGBM$^{7}$, XGBoost$^{8}$, and CatBoost$^{10}$

### Main Idea

Gradient boosting main idea is to train next iteration model on residual of previous model as the target. 

In [125]:
from sklearn.ensemble import GradientBoostingRegressor

In [126]:
# TODO define GradientBoostingRegressor, n_estimators=10
mdl = 

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=10,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [127]:
# TODO: do_cv on GradientBoostingRegressor

Cross Validation 1

    ....train metrics....
    rmse: 55.47491107519263
    mae: 47.931418569695666,
    r2: 0.5065771601937898
    

    ....test metrics....
    rmse: 52.56444394990826
    mae: 43.73903490279989,
    r2: 0.40242178236903403
    
Cross Validation 2

    ....train metrics....
    rmse: 53.35330434322521
    mae: 45.85119284616699,
    r2: 0.5109397979959944
    

    ....test metrics....
    rmse: 63.39114366471927
    mae: 53.26895554133836,
    r2: 0.3527880519487061
    
Cross Validation 3

    ....train metrics....
    rmse: 53.289577662852885
    mae: 45.35429693697925,
    r2: 0.5247353629381997
    

    ....test metrics....
    rmse: 62.3771317980767
    mae: 53.06529940099563,
    r2: 0.3111134045227266
    
Cross Validation 4

    ....train metrics....
    rmse: 53.39010855604922
    mae: 45.41101696828909,
    r2: 0.5072482289524957
    

    ....test metrics....
    rmse: 62.81946994086882
    mae: 53.1134645599001,
    r2: 0.38501160269023027
    
Cross 

# Stacking

Stacking is another *Meta - Feature* learning algorithm that "stack" models together. 

## Main Idea
Stacking works by constructing network of models. We learn to *weight* each prediction result to construct final, more accurate prediction.

![stacking](https://cdn-images-1.medium.com/max/1600/0*GHYCJIjkkrP5ZgPh.png)


In [128]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

In [129]:
class StackModel(object):
    def __init__(self, models):
        # TODO: define models and meta_learner as linear regression
        self.models = 
        self.meta_learner = 
        
    def fit(self, X, y):
        predictions = list()
        # TODO: define 
    
        return self
    
    def predict(self, X):
        predictions = list()
        # TODO: define and return final_predictions
        
        return final_prediction
    

In [130]:
models = [
    # make pipeline for each model, containing minmaxscaler as preprocessor and the model
    
]

In [131]:
mdl = # TODO

In [132]:
# TODO: do_cv

Cross Validation 1

    ....train metrics....
    rmse: 27.526870386874396
    mae: 20.95588714658343,
    r2: 0.8785102341960441
    

    ....test metrics....
    rmse: 62.90063010602655
    mae: 49.449384572691606,
    r2: 0.14430183360388305
    
Cross Validation 2

    ....train metrics....
    rmse: 25.420983563466706
    mae: 19.170406795039924,
    r2: 0.88897408044089
    

    ....test metrics....
    rmse: 58.37330465132906
    mae: 46.33887812408529,
    r2: 0.4511951853681898
    
Cross Validation 3

    ....train metrics....
    rmse: 26.087357228349216
    mae: 20.07656384004358,
    r2: 0.8861033068968689
    

    ....test metrics....
    rmse: 68.81116456051153
    mae: 52.91720167545122,
    r2: 0.1616705027175802
    
Cross Validation 4

    ....train metrics....
    rmse: 26.71528532801037
    mae: 19.571672707095523,
    r2: 0.8766252686460366
    

    ....test metrics....
    rmse: 66.74097407750051
    mae: 56.96343240600481,
    r2: 0.30583379142640654
    
Cr

# References

[1] Schapire, Robert E. [*Explaining AdaBoost*](http://rob.schapire.net/papers/explaining-adaboost.pdf). Princeton University

[2] Freund, Yoav & Schapire, Robert E. [*A Short Introduction to Boosting*](https://cseweb.ucsd.edu/~yfreund/papers/IntroToBoosting.pdf). AT&T Labs

[3] Sicotte, Xavier Bourret. [*Adaboost: Implementation and Intuition*](https://xavierbourretsicotte.github.io/AdaBoost.html)

[4] Drucker, Harris. [*Improving Regressors using Boosting Techniques*](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.31.314&rep=rep1&type=pdf)

[5] Wikipedia. [Bootstraping (Statistics)](https://en.wikipedia.org/wiki/Bootstrapping_(statistics))

[6] Breiman, Leo. [*Bagging Predictors*](https://www.stat.berkeley.edu/~breiman/bagging.pdf). 1994. Technical Report No.421

[7] Ke, Guolin et al. [*LightGBM: A Highly Efficient Gradient Boosting Decision Tree*](https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf). 2017. NIPS

[8] Chen, Tianqi & Guestrin, Carlos. [*XGBoost: A Scalable Tree Boosting System*](https://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf). 2016. KDD

[9] Friedman, Jerome H. [*Greedy Function Approximation: A Gradient Boosting Machine*](https://statweb.stanford.edu/~jhf/ftp/trebst.pdf). 1999. IMS 1999 Reitz Lecture.

[10] Prokhorenkova, Liudmila et al. [*CatBoost: unbiased boosting with categorical features*](https://arxiv.org/pdf/1706.09516.pdf). 2019. ArXiv.

[11] Wolpert, David H. [*Stacked Generalization*](https://www.sciencedirect.com/science/article/pii/S0893608005800231). 1992. Neural Networks